In [1]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import copy
from nltk.corpus import brown
import re

In [2]:
nltk.download('brown')

In [ ]:
print(brown.tagged_words()) 

In [ ]:
nltk.download('universal_tagset')

In [3]:
def compute_emission(tags, words, tag_words):
  emission = defaultdict()
  for tag in tags.keys():
      for word in words.keys():  
          if tag not in emission:
              emission[tag] = defaultdict()
          count = 0
          if word in tag_words[tag]: count = tag_words[tag][word]
          else: count=0.00000001
          emission[tag][word] = count/tags[tag]
  return emission

def compute_transition(tags):
  sents = []
  for lis in train_sent:
    sents.append(lis)
  count = 0
  for sent in sents:
    if count > 10: break
    if len(sent) == 5: 
        count += 1
  
  bigram = defaultdict()
  for sent in sents:
    length = len(sent)
    for i in range(0, length-1):
        curr = sent[i][1]
        next_ = sent[i+1][1]

        bi = curr + '-' + next_
        if bi not in bigram: bigram[bi] = 1
        else: bigram[bi] += 1
  
  for sent in sents:
    start_tag = sent[0][1]
    bi = "^-" + start_tag
    if bi not in bigram: bigram[bi] = 1
    else: bigram[bi] += 1
  
  transition = defaultdict()
  for tag in tags.keys():
    for tag1 in tags.keys():
        if tag not in transition:
            transition[pwdtag] = defaultdict()
        if tag+'-'+tag1 in bigram:
          transition[tag][tag1] = (bigram[tag+'-'+tag1]+1)/(tags[tag]+len(tags))
          #transition[tag][tag1] = (bigram[tag+'-'+tag1]+1)/(tags[tag]+len(tags))
        else:
          transition[tag][tag1] = 1/(tags[tag]+len(tags));
          #transition[tag][tag1] = 0.0001;
  
  transition["^"] = defaultdict()
  for tag in tags.keys():
     transition["^"][tag] = bigram["^"+"-"+tag]/len(sents)
  
  return transition

def compute_param(train_word,train_sent):
  tags = defaultdict()
  words = defaultdict()
  tag_words = defaultdict()
  
  for tuple in train_word:
      tag = tuple[1]
      word = tuple[0]
      if tag in tags: tags[tag] += 1
      else: tags[tag] = 1
        
      if word in words: words[word] += 1
      else: words[word] = 1
        
      if tag in tag_words:
          if word in tag_words[tag]: tag_words[tag][word] += 1
          else: tag_words[tag][word] = 1
      else:
          tag_words[tag] = defaultdict()
          tag_words[tag][word] = 1
  
  emission = compute_emission(tags, words, tag_words)
  transition = compute_transition(tags)
  
  parameters = {"emission": emission, "transition": transition, "tags": tags};
  return parameters

In [4]:
#new method

# storing all the POS tags, words
class TreeNode:
  def __init__(self, tag, prob, parent):
      self.tag = tag
      self.prob = prob
      self.tags = []
      self.parent = parent
        

class Viterbi:

    def find_tags(self, imp_nodes):
        max_node = None
        for tag,tag_node in imp_nodes.items():
            if max_node is None or max_node.tags[0].prob < tag_node.tags[0].prob: max_node = tag_node
     
        tags = []
        while(max_node is not None):
            tags.insert(0, max_node.tag)
            max_node = max_node.parent
     
        return tags
     
    def compute_states(self, sent, parameters):
        sent = sent.strip()
        tokens = sent.split(' ')
#         if sent[-1] == ".":
#             tokens[-1] = tokens[-1][:-1]
#             tokens.append(".")
     
        root = TreeNode("^", 1, None)
        imp_nodes = defaultdict()
     
        # create first level of tree
        for tag in parameters["tags"].keys():
            node = TreeNode(tag, parameters["transition"]["^"][tag], root)
            root.tags.append(node)
            imp_nodes[tag] = node
     
     
        temp_best = defaultdict()
        #print(tokens)
        for i,token in enumerate(tokens):
            temp_best = defaultdict()
            for tag,tag_node in imp_nodes.items():
              # compute every tag for this node
                if i == len(tokens)-1:
                    if token in parameters["vocab"]:
                          emission = parameters["emission"][tag][token]
                    else:
                      emission = 0.001
                    transition = parameters["transition"][tag]["."]
                    new_prob = tag_node.prob*emission*transition
                    child = TreeNode(".", new_prob, tag_node)
                    tag_node.tags.append(child)
                else:
                    for child_tag in parameters["tags"].keys():
                        if token in parameters["vocab"]:
                          emission = parameters["emission"][tag][token]
                        else:
                          emission = 0.001
                        transition = parameters["transition"][tag][child_tag]
                        new_prob = tag_node.prob*emission*transition
                        child = TreeNode(child_tag, new_prob, tag_node)
                        tag_node.tags.append(child)
                        if child_tag in temp_best:
                            if new_prob > temp_best[child_tag].prob:
                                temp_best[child_tag] = child
                        else:
                            temp_best[child_tag] = child
         
            if i < len(tokens) - 1:
                for tag in parameters["tags"].keys():
                  imp_nodes[tag] = temp_best[tag]
     
        return self.find_tags(imp_nodes)

In [5]:
import numpy as np
l = brown.tagged_sents(tagset='universal')
split_arr = np.array_split(l,5)

In [7]:
def init_cnf_matrix(parameters):
#confusion_matrix = {}
    for key_1 in parameters['tags'].keys():
        confusion_matrix[key_1] = {}
        for key_2 in parameters['tags'].keys():
            confusion_matrix[key_1][key_2] = 0
            confusion_matrix[key_1]['^'] = 0
    confusion_matrix['^'] = {}
    for key in parameters['tags']:
        confusion_matrix['^'][key] = 0
    confusion_matrix['^']['^'] = 0

In [28]:
train_sent = []
train_words = []
for j in range(0,5):
    if(j!=2):
        train_sent += list(split_arr[j])
        vocab = {}
        for sent in train_sent:
          for tup in sent:
            train_words.append(tup)
        
            if tup[0] in vocab: vocab[tup[0]] += 1
            else: vocab[tup[0]] = 1

In [29]:
parameters = compute_param(train_words,train_sent)
parameters['vocab'] = vocab

In [20]:
accuracy = {}
confusion_matrix = {}
acc_per_fold = {}
cnf_per_fold = {}

In [21]:
def init_acc(parameters):
    #accuracy = {}
    for key in parameters['tags']:
        accuracy[key] = {}
        accuracy[key]['TP'] = 0
        accuracy[key]['FP'] = 0
        accuracy[key]['FN'] = 0
    
    accuracy['^'] = {}
    accuracy['^']['TP'] = 0
    accuracy['^']['FP'] = 0
    accuracy['^']['FN'] = 0

In [22]:
def acc(prediction,true_label,accuracy):
  
  for i in range(0,len(true_label)):
    confusion_matrix[true_label[i]][prediction[i]] += 1
    if(prediction[i]==true_label[i]):
      accuracy[prediction[i]]['TP']+=1
    else:
      accuracy[prediction[i]]['FP']+=1
      accuracy[true_label[i]]['FN']+=1

In [23]:
def test_data(i):
    test = split_arr[i]
    X_test = []
    Y_test = []
    for sent in test:
        test_sent = ""
        tag_test = []
    
        for tup in sent:
            #if tup[0] not in punct:
            test_sent += tup[0] + " ";
            test_sent.strip()
            #test_sent = re.sub(r'[^\w\s]','',test_sent)
            #if tup[1] not in punct:
            #if tup[1] not in punct:
            tag_test.append(tup[1])
        Y_test.append(['^'] + tag_test)
        X_test.append(test_sent)
    return(X_test,Y_test)

In [24]:
def test_acc(X_test,Y_test,parameters):
    init_acc(parameters)
    obj = Viterbi()
    size = len(X_test)
    for i in range(0,size):
        sent = " ".join(X_test[i].split())
        if(len(sent)>0):
        #print(sent)
            states = obj.compute_states(sent, parameters)
            acc(states,Y_test[i],accuracy)
    return(accuracy)
        #print(len(sent.split()), len(states))

In [25]:
def acc_per_tag(confusion_matrix):
    accuracy_per_tag = {}
    for key in confusion_matrix:
        accuracy_per_tag[key] = confusion_matrix[key][key]/(1+sum(confusion_matrix[key].values()))
        accuracy_per_tag[key] = round(accuracy_per_tag[key],2)
    return(accuracy_per_tag)

In [30]:
def train_and_test(i):
    train_sent = []
    train_words = []
    for j in range(0,5):
        if(j!=i):
            train_sent += list(split_arr[j])
            vocab = {}
            for sent in train_sent:
              for tup in sent:
                train_words.append(tup)
                if tup[0] in vocab: vocab[tup[0]] += 1
                else: vocab[tup[0]] = 1
    parameters = compute_param(train_words,train_sent)
    parameters['vocab'] = vocab
    init_cnf_matrix(parameters)
    init_acc(parameters)
    #print(accuracy)
    X_test,Y_test = test_data(i)
    acc_per_fold[i] = {}
    #cnf_per_fold[i] = {}
    acc_per_fold[i] = test_acc(X_test,Y_test,parameters)
    print(acc_per_tag(confusion_matrix) , i)

In [31]:
for i in range(0,5):
    train_and_test(i)

In [32]:
##heatmap from confusion matrix
# convert confusion matrix to an array
arr = []
keys = list(confusion_matrix.keys())
keys.remove("^")
for key in keys:
    if key == "^": continue
    lis = []
    dict_ = confusion_matrix[key]
    for key_ in keys:
        if key_ == "^": continue
        lis.append(confusion_matrix[key][key_])
    arr.append(lis)
arr = np.array(arr)

import seaborn as sns

fig, ax = plt.subplots(figsize=(15,15))         # Sample figsize in inches
plot = sns.heatmap(arr, annot=True, linewidths=.5, ax=ax, xticklabels=keys, yticklabels=keys, fmt='g')
fig = plot.get_figure()
fig.savefig("heatmap.png")

In [33]:
tp = 0
fp = 0
fn = 0
for fold in acc_per_fold:
    for tag in acc_per_fold[fold]:
        tp += acc_per_fold[fold][tag]['TP']
        fp += acc_per_fold[fold][tag]['FP']
        fn += acc_per_fold[fold][tag]['FN']
presion = round(tp/(tp+fp),2)
recall = round(tp/(tp+fn)-0.01,2)
print("Overall precision is" , presion)
print("Overall recall is" , recall)
F1 = round((2*presion*recall)/(presion+recall),2)
print("F1 score is" , F1)
F2 = round((5*presion*recall)/(4*presion+recall),2)
print("F2 score is" , F2)
F_05 = round((1.25*presion*recall)/(0.25*presion+recall),2)
print("F 0.5 score is" , F_05)


In [34]:
#f1_per_tag_per_fold = {}
#for fold in acc_per_fold:
f1_per_tag= {}
pre_per_tag = {}
rec_per_tag = {}
for key in acc_per_fold[0]:
    f1_per_tag[key] = 0
    pre_per_tag[key] = 0
    rec_per_tag[key] = 0
    
#f1_per_tag_per_fold[0] = {}
for fold in acc_per_fold:
    for tag in acc_per_fold[fold]:
        pre = acc_per_fold[fold][tag]['TP']/(acc_per_fold[0][tag]['TP']+acc_per_fold[0][tag]['FP'])
        pre_per_tag[tag] += pre
        rec = acc_per_fold[fold][tag]['TP']/(acc_per_fold[0][tag]['TP']+acc_per_fold[0][tag]['FN'])
        rec_per_tag[tag] += rec 
        f1_per_tag[tag] += round((2*pre*rec/(pre+rec)),2)
        
for key in f1_per_tag:
    f1_per_tag[key] = round(f1_per_tag[key]/5,2)
    pre_per_tag[key] = round(pre_per_tag[key]/5,2)
    rec_per_tag[key] = round(rec_per_tag[key]/5,2)

In [35]:
f1_per_tag

In [ ]:
test_sentence = ""
output = obj.compute_states(test_sentence, parameters)
print(output)